In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
class ParallelCNNRNN(nn.Module):
    def __init__(self, num_classes=8):
        super(ParallelCNNRNN, self).__init__()
        
        self.nb_filters1 = 16
        self.nb_filters2 = 32
        self.nb_filters3 = 64
        self.nb_filters4 = 64
        self.nb_filters5 = 64
        self.ksize = (3, 1)
        self.pool_size_1 = (2, 2)
        self.pool_size_2 = (4, 4)
        self.pool_size_3 = (4, 2)
        self.lstm_count = 64

        self.conv1 = nn.Sequential(
            nn.Conv2d(1, self.nb_filters1, kernel_size=self.ksize),
            nn.ReLU(),
            nn.MaxPool2d(self.pool_size_1)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(self.nb_filters1, self.nb_filters2, kernel_size=self.ksize),
            nn.ReLU(),
            nn.MaxPool2d(self.pool_size_1)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(self.nb_filters2, self.nb_filters3, kernel_size=self.ksize),
            nn.ReLU(),
            nn.MaxPool2d(self.pool_size_1)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(self.nb_filters3, self.nb_filters4, kernel_size=self.ksize),
            nn.ReLU(),
            nn.MaxPool2d(self.pool_size_2)
        )
        self.conv5 = nn.Sequential(
            nn.Conv2d(self.nb_filters4, self.nb_filters5, kernel_size=self.ksize),
            nn.ReLU(),
            nn.MaxPool2d(self.pool_size_2)
        )
        
        self.cnn_flatten_dim = 256 

        self.pool_lstm = nn.MaxPool2d(self.pool_size_3)
        
        self.gru = nn.GRU(input_size=64, hidden_size=self.lstm_count, 
                          bidirectional=True, batch_first=True)

        self.fc = nn.Linear(self.cnn_flatten_dim + (self.lstm_count * 2), num_classes)

    def forward(self, x):
        c = self.conv1(x)
        c = self.conv2(c)
        c = self.conv3(c)
        c = self.conv4(c)
        c = self.conv5(c)
        
        c_flat = c.view(c.size(0), -1)
        
        r = self.pool_lstm(x)
        
        r = r.squeeze(1) 
        
        r_out, _ = self.gru(r)
        r_last = r_out[:, -1, :]
        
        combined = torch.cat((c_flat, r_last), dim=1)
        
        out = self.fc(combined)
        
        return out

In [3]:
X_train_torch = torch.randn(100, 1, 640, 128)
y_train_torch = torch.randint(0, 8, (100,))
X_valid_torch = torch.randn(20, 1, 640, 128)
y_valid_torch = torch.randint(0, 8, (20,))

BATCH_SIZE = 32
train_dataset = TensorDataset(X_train_torch, y_train_torch)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

valid_dataset = TensorDataset(X_valid_torch, y_valid_torch)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [4]:
model = ParallelCNNRNN(num_classes=8).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters(), lr=0.0005)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='max', factor=0.5, patience=10, threshold=0.01, verbose=True
)

C:\Users\dhgml\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [5]:
EPOCHS = 50
best_acc = 0.0

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
    train_acc = correct / total
    
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for inputs, targets in valid_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_total += targets.size(0)
            val_correct += predicted.eq(targets).sum().item()
            
    val_acc = val_correct / val_total
    
    scheduler.step(val_acc)
    
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), 'weights.best.pth')
        print(f" -> Model saved! (Val Acc: {val_acc:.4f})")
        
    print(f"Train Loss: {train_loss/len(train_loader):.4f} | Train Acc: {train_acc:.4f}")
    print(f"Val Loss: {val_loss/len(valid_loader):.4f} | Val Acc: {val_acc:.4f}")
    print("-" * 60)

Epoch 1/50: 100%|██████████| 4/4 [00:01<00:00,  3.88it/s]


 -> Model saved! (Val Acc: 0.0500)
Train Loss: 2.1296 | Train Acc: 0.1500
Val Loss: 2.1715 | Val Acc: 0.0500
------------------------------------------------------------


Epoch 2/50: 100%|██████████| 4/4 [00:00<00:00, 41.00it/s]


 -> Model saved! (Val Acc: 0.3000)
Train Loss: 2.0768 | Train Acc: 0.1600
Val Loss: 2.0965 | Val Acc: 0.3000
------------------------------------------------------------


Epoch 3/50: 100%|██████████| 4/4 [00:00<00:00, 42.03it/s]


...

Train Loss: 1.6231 | Train Acc: 0.4600
Val Loss: 2.0581 | Val Acc: 0.2000
------------------------------------------------------------


Epoch 49/50: 100%|██████████| 4/4 [00:00<00:00, 42.10it/s]


Train Loss: 1.6760 | Train Acc: 0.4600
Val Loss: 2.0601 | Val Acc: 0.2000
------------------------------------------------------------


Epoch 50/50: 100%|██████████| 4/4 [00:00<00:00, 41.12it/s]

Train Loss: 1.6190 | Train Acc: 0.4600
Val Loss: 2.0627 | Val Acc: 0.2000
------------------------------------------------------------
